In [45]:
import generate_data as vaina
import models
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder
from evaluate import *


import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

Sacamos el conjunto de datos de entramiento y lo dividimos en 5% del subset para entrenamiento

In [46]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

data = vaina.Data(layer_size=6)
train, val, test = data.get_splits(['random_subsample'], [[0.05, 0.95]])
train_unshuffled = train.copy()
np.random.shuffle(train)

Number of samples: 26 train | 100 val | 403 test


c:\Users\juand\anaconda3\lib\site-packages\sklearn\base.py:443: UserWarning: X has feature names, but OneHotEncoder was fitted without feature names
  warnings.warn(


Separamos todos los datos en x (variables), y (clasificacion) 

In [47]:
x_train, y_train = data.get_x_y(train)
x_val, y_val = data.get_x_y(val)
x_test, y_test = data.get_x_y(test)

In [48]:
train_original = data.reverse_encoding(data.get_x_y(train_unshuffled)[0])
val_original = data.reverse_encoding(x_val)
test_original = data.reverse_encoding(x_test)

print("--------train_original--------", '\n')
pos_train_ratio = get_stats_and_ratio(train_original)
print("--------test_original--------", '\n')
pos_test_ratio = get_stats_and_ratio(test_original)
print("--------val_original--------", '\n')
pos_val_ratio = get_stats_and_ratio(val_original)

--------train_original-------- 

Number of samples by case:
F3 dr:    F2-wu 1 | F2-uw 2 | F2-w 1 | F2-noop 2 | 
F3 cr:    F2-wu 2 | F2-uw 1 | F2-w 3 | F2-noop 1 | 
F3 r:    F2-wu 0 | F2-uw 1 | F2-w 1 | F2-noop 4 | 
F3 noop:    F2-wu 3 | F2-uw 1 | F2-w 2 | F2-noop 1 | 

Positive samples count: 11
Total samples count: 26
Positive class ratio: 0.4230769230769231
--------test_original-------- 

Number of samples by case:
F3 dr:    F2-wu 20 | F2-uw 17 | F2-w 24 | F2-noop 22 | 
F3 cr:    F2-wu 20 | F2-uw 21 | F2-w 22 | F2-noop 27 | 
F3 r:    F2-wu 27 | F2-uw 25 | F2-w 30 | F2-noop 29 | 
F3 noop:    F2-wu 24 | F2-uw 23 | F2-w 32 | F2-noop 40 | 

Positive samples count: 174
Total samples count: 403
Positive class ratio: 0.4317617866004963
--------val_original-------- 

Number of samples by case:
F3 dr:    F2-wu 4 | F2-uw 6 | F2-w 5 | F2-noop 11 | 
F3 cr:    F2-wu 3 | F2-uw 3 | F2-w 5 | F2-noop 7 | 
F3 r:    F2-wu 3 | F2-uw 4 | F2-w 5 | F2-noop 9 | 
F3 noop:    F2-wu 8 | F2-uw 11 | F2-w 8 | F2-

Mandamos las etiquetas codificadas al dispositivo del tensor

In [49]:
# Crear un codificador de etiquetas
label_encoder = LabelEncoder()

# Ajustar el codificador a las etiquetas y transformarlas
y_train = label_encoder.fit_transform(y_train)
y_val = label_encoder.transform(y_val)
y_test = label_encoder.transform(y_test)

# Convertir las etiquetas a tensores de PyTorch
y_train = torch.tensor(y_train, dtype=torch.float).to(device)
y_val = torch.tensor(y_val, dtype=torch.float).to(device)
y_test = torch.tensor(y_test, dtype=torch.float).to(device)

In [50]:
x_train_cnn = data.to_conv_format(x_train)
x_val_cnn = data.to_conv_format(x_val)
x_test_cnn = data.to_conv_format(x_test)
for i in range(len(x_train_cnn)):
    x_train_cnn[i] = x_train_cnn[i].to(device)
    x_val_cnn[i] = x_val_cnn[i].to(device)
    x_test_cnn[i] = x_test_cnn[i].to(device)

In [51]:
num_experiments = 6
epochs = 200
early_stopping_limit = 100

experiment_name="25per"

In [63]:
def train_models(constructor, x_train, x_val, x_test, weight_decay, *argv):
    accuracies = []
    precisions = []
    recalls = []
    f1_scores = []
    wrong_preds = []
    best_accuracy = 0
    
    train_losses = []
    val_losses = []
    train_accs = []
    val_accs = []

    for i in range(num_experiments):
        model = constructor(*argv)
        model.to(device)

        criterion = nn.CrossEntropyLoss()
        optimizer = optim.Adam(model.parameters(), weight_decay=weight_decay)

        train_losses.append([])
        val_losses.append([])
        train_accs.append([])
        val_accs.append([])
        
        best_acc = 0

        early_stopping_cnt = 0

        for epoch in range(1, epochs + 1):
            train_loss, train_acc = models.train_epoch(model, x_train, y_train, criterion, optimizer, epoch, 10, verbose=False)
            val_loss, val_acc = models.eval_epoch(model, x_val, y_val, criterion, 'Validation', verbose=False)

            
            train_losses[-1].append(train_loss)
            val_losses[-1].append(val_loss)
            train_accs[-1].append(train_acc)
            val_accs[-1].append(val_acc)
            
            model_name = constructor.__name__[:constructor.__name__.find('')]
            if val_acc > best_acc:
                best_acc = val_acc
                torch.save(model.state_dict(), f'./{model_name}model_TEMP' + experiment_name)    
                early_stopping_cnt = 0
            else:
                early_stopping_cnt += 1

            if early_stopping_cnt >= early_stopping_limit:
                break


        model.load_state_dict(torch.load(f'./{model_name}model_TEMP' + experiment_name))
    
        accuracies.append(get_accuracy_by_cases(model, x_test, y_test, test_original))
     #   precisions.append(get_precision_by_cases(model, x_test, y_test, test_original))
      #  recalls.append(get_recall_by_cases(model, x_test, y_test, test_original))
       # f1_scores.append(get_f1_by_cases(model, x_test, y_test, test_original))
        
        wrong_preds.append(get_wrong_predictions(model, x_test, y_test, test_original))

        if accuracies[-1]['Overall'] > best_accuracy:
            torch.save(model.state_dict(), f'./best_{model_name}model' + experiment_name)    
            best_accuracy = accuracies[-1]['Overall']

        print(i + 1, "/", num_experiments, "models trained | Current model test accuracy:", accuracies[-1]['Overall'])
    return accuracies, wrong_preds, [train_losses, val_losses, train_accs, val_accs]

In [64]:
cnn_accuracies, cnn_wrong_preds, cnn_epoch_stats = train_models(models.CNN_Model, x_train_cnn, x_val_cnn, x_test_cnn, 0.0001, data, 64, 128, 4, -1, 'keras')

C:\Users\juand\AppData\Local\Temp\ipykernel_15200\3672781297.py:52: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load(f'./{model_name}model_TEMP

1 / 6 models trained | Current model test accuracy: 0.5682382133995038
2 / 6 models trained | Current model test accuracy: 0.6327543424317618
3 / 6 models trained | Current model test accuracy: 0.9503722084367245
4 / 6 models trained | Current model test accuracy: 0.9578163771712159
5 / 6 models trained | Current model test accuracy: 0.913151364764268
6 / 6 models trained | Current model test accuracy: 0.4466501240694789


In [32]:
x_train_lstm = data.to_lstm_format(x_train)
x_val_lstm = data.to_lstm_format(x_val)
x_test_lstm = data.to_lstm_format(x_test)
for i in range(len(x_train_lstm)):
    x_train_lstm[i] = x_train_lstm[i].to(device)
    x_val_lstm[i] = x_val_lstm[i].to(device)
    x_test_lstm[i] = x_test_lstm[i].to(device)

In [33]:
lstm_accuracies, lstm_wrong_preds, lstm_epoch_stats = train_models(models.LSTM_Model, x_train_lstm, x_val_lstm, x_test_lstm, 0.0001, data, 16, 32, 8, 'keras')

C:\Users\juand\AppData\Local\Temp\ipykernel_15200\1066353585.py:49: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load(f'./{model_name}_model_TEM

1 / 6 models trained | Current model test accuracy: 0.9578163771712159
2 / 6 models trained | Current model test accuracy: 0.9578163771712159
3 / 6 models trained | Current model test accuracy: 0.9578163771712159
4 / 6 models trained | Current model test accuracy: 0.9578163771712159
5 / 6 models trained | Current model test accuracy: 0.9578163771712159
6 / 6 models trained | Current model test accuracy: 0.9578163771712159


In [34]:
x_train_normal = data.to_conv_format(x_train)
x_val_normal = data.to_conv_format(x_val)
x_test_normal = data.to_conv_format(x_test)
for i in range(len(x_train_normal)):
    x_train_normal[i] = x_train_normal[i].to(device)
    x_val_normal[i] = x_val_normal[i].to(device)
    x_test_normal[i] = x_test_normal[i].to(device)

In [35]:
deepsetv2_accuracies, deepsetv2_wrong_preds, deepsetv2_epoch_stats = train_models(models.DEEPSETV2_Model, x_train_normal, x_val_normal, x_test_normal, 0.0001, data, 32, 8, 'keras')

C:\Users\juand\AppData\Local\Temp\ipykernel_15200\1066353585.py:49: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load(f'./{model_name}_model_TEM

1 / 6 models trained | Current model test accuracy: 0.9578163771712159
2 / 6 models trained | Current model test accuracy: 0.9578163771712159
3 / 6 models trained | Current model test accuracy: 0.9578163771712159
4 / 6 models trained | Current model test accuracy: 0.9578163771712159
5 / 6 models trained | Current model test accuracy: 0.9578163771712159
6 / 6 models trained | Current model test accuracy: 0.9578163771712159


In [36]:
feedforward_accuracies, feedforward_wrong_preds, feedforward_epoch_stats = train_models(models.FEEDFORWARD_Model, x_train_normal, x_val_normal, x_test_normal, 0.0001, data, 128, 32, 8, 'keras')

C:\Users\juand\AppData\Local\Temp\ipykernel_15200\1066353585.py:49: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load(f'./{model_name}_model_TEM

1 / 6 models trained | Current model test accuracy: 0.9553349875930521
2 / 6 models trained | Current model test accuracy: 0.9578163771712159
3 / 6 models trained | Current model test accuracy: 0.9578163771712159
4 / 6 models trained | Current model test accuracy: 0.9578163771712159
5 / 6 models trained | Current model test accuracy: 0.9578163771712159
6 / 6 models trained | Current model test accuracy: 0.9578163771712159
